In [ ]:
# Figure out rotations correctly.

import roboticstoolbox as rtb
from spatialmath import SE3
import numpy as np
import imageio

# Definition of the Lego robot arm structure (links and joints). This is a right arm:
studs = 0.008  # 8 mm between Lego studs
# Define joint limits in degrees and convert to radians
joint_limits_deg = [
    [-30, 85],    # Joint 1: +45 to -10 degrees
    [0, 85],      # Joint 2: +35 to 0 degrees  
    [-89, 89],   # Joint 3: +35 to -120 degrees
    [-89, 89],    # Joint 4: +13 to -20 degrees
    [-120, 120],  # Joint 5: +120 to -120 degrees
    [-110, 110]   # Joint 6: +110 to -110 degrees
]
joint_limits_rad = np.deg2rad(joint_limits_deg)

E1 = rtb.ET.tz(0.377)                                 # 37.7 cm up
E2 = rtb.ET.Ry(flip=True, qlim=joint_limits_rad[0])   # Shoulder rotation forward/back                 (positive angle is forwards)
E3 = rtb.ET.ty(-9.5*studs)                            # 9.5 studs out to next shoulder joint
E4 = rtb.ET.Rx(flip=True, qlim=joint_limits_rad[1])   # Shoulder rotation out/up / in/down to the side (positive angle is out/up)
E5 = rtb.ET.tz(-20.5*studs)                           # 20.5 studs down to overarm rotation joint
E6 = rtb.ET.Rz(qlim=joint_limits_rad[2])              # Overarm rotation                               (positive angle is counter clockwise in towards the body)
E7 = rtb.ET.tz(-12.5*studs)                           # 12.5 studs down to elbow
E8 = rtb.ET.Ry(flip=True, qlim=joint_limits_rad[3])   # Elbow rotation                                 (positive angle is flexing the elbow upwards)
E9 = rtb.ET.tx(10.5*studs)                            # 10.5 studs out to underarm rotation joint
E10 = rtb.ET.Rx(qlim=joint_limits_rad[4])             # Underarm rotation                              (positive angle is rotating the underarm clockwise)
E11 = rtb.ET.tx(23.5*studs)                           # 23.5 studs out to wrist rotation joint
E12 = rtb.ET.Rz(qlim=joint_limits_rad[5])             # Wrist rotation up/down                         (positive angle is rotating the wrist downwards/inwards)
E13 = rtb.ET.tx(7*studs)                              # 7 studs out to gripper (TODO: Fix actual length))

robot_arm = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13
#print(robot_arm) # View the ETS (Elementary Transformation Sequence, where ET is a translation or rotation).
#print(f"The robot has {robot_arm.n} joints")
#print(f"The robot has {robot_arm.m} Elementary Transformations (ETs)")

neutral_pose_with_bent_elbow_q = np.array([0, 0, 0, 0, 0, 0])
neutral_pose_se3 = robot_arm.fkine(neutral_pose_with_bent_elbow_q) # Calculate the forward kinematics of the neutral pose with a bent elbow
print(f"\nneutral_pose_with_bent_elbow: \n{neutral_pose_se3}")
#print(f"Rotation around the Z, Y and Z axes for this pose: {neutral_pose_se3.eul(unit='deg')}")

elbow_extended_q = np.deg2rad(np.array([0, 0, 0, -90, 0, 0]))
elbow_extended_se3 = robot_arm.fkine(elbow_extended_q) 
#print(f"elbow_extended: \n{elbow_extended_se3}")
#print(f"Rotation around the Z, Y and Z axes for this pose: {elbow_extended_se3.eul(unit='deg')}")

# Definition of rotation matrix for conversion from 
# Unity's coordinate system (x-right, y-up, z-forward)
# to the robot arm's coordinate system (x-forward, y-left, z-up):
# Fixed to be a proper rotation matrix (determinant = +1)
unity_to_robot_rotation = np.array([
    [0,  0, 1, 0],   # Unity z (forward) → Robot x (forward)
    [-1, 0, 0, 0],   # Unity x (right) → Robot -y (left)
    [0,  1, 0, 0],    # Unity y (up) → Robot z (up)
    [0,  0, 0, 1]     
]).astype(np.float64)  # Use float64 for higher precision
unity_to_robot_rotation_inv = np.linalg.inv(unity_to_robot_rotation)
#print(f"unity_to_robot_rotation: \n{unity_to_robot_rotation}")
#print(f"unity_to_robot_rotation_inv: \n{unity_to_robot_rotation_inv}")
#print(f"unity_to_robot_rotation_inv has type: {unity_to_robot_rotation_inv.dtype}")
unity_postion_offset = np.array([0.25, -0.40, 0.30])    # Offset in meters (in Unity starts 25cm right, 
                                                        # 40cm down from eye level and 30cm forward)
robot_postion_offset = np.array([0.328, -0.076, 0.113]) # Offset in meters (physical robot arm starts 32.8cm forward, 
                                                        # 7,6cm right and 11.3cm up from table level)

se3_list = []

unity_pos_array_4x4 = np.array([[1,     0,    0,      0.25],
                                [0,     1,    0,     -0.40],
                                [0,     0,    1,      0.30],
                                [0,     0,    0,      1   ]])
#print(f"unity_pos_array_4x4: \n{unity_pos_array_4x4}")
translated_unity_pos = unity_pos_array_4x4
translated_unity_pos[:3, 3] -= unity_postion_offset  # Subtract the Unity offset to the position
print(f"translated_unity_pos: \n{translated_unity_pos}")
pos_in_robot_coordinate_system = np.dot( np.dot(unity_to_robot_rotation, unity_pos_array_4x4), unity_to_robot_rotation_inv)
#print(f"pos_in_robot_coordinate_system: \n{pos_in_robot_coordinate_system}")
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (before applying offset): \n{pos_in_robot_coordinate_system_se3}")
pos_in_robot_coordinate_system[:3, 3] += robot_postion_offset  # Add the robot coordinate system offset to the position
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (after applying offset): \n{pos_in_robot_coordinate_system_se3}")
#se3_list.append(pos_in_robot_coordinate_system_se3)


unity_pos_array_4x4 = np.array([[1,     0,    0,      0.35],
                                [0,     1,    0,     -0.20],
                                [0,     0,    1,      0.60],
                                [0,     0,    0,      1   ]])
#print(f"unity_pos_array_4x4: \n{unity_pos_array_4x4}")
translated_unity_pos = unity_pos_array_4x4
translated_unity_pos[:3, 3] -= unity_postion_offset  # Subtract the Unity offset to the position
print(f"translated_unity_pos: \n{translated_unity_pos}")
pos_in_robot_coordinate_system = np.dot( np.dot(unity_to_robot_rotation, unity_pos_array_4x4), unity_to_robot_rotation_inv)
#print(f"pos_in_robot_coordinate_system: \n{pos_in_robot_coordinate_system}")
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (before applying offset): \n{pos_in_robot_coordinate_system_se3}")
pos_in_robot_coordinate_system[:3, 3] += robot_postion_offset  # Add the robot coordinate system offset to the position
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (after applying offset): \n{pos_in_robot_coordinate_system_se3}")
#se3_list.append(pos_in_robot_coordinate_system_se3)


unity_pos_array_4x4 = np.array([[1,     0,    0,      0.25],
                                [0,     1,    0,     -0.40],
                                [0,     0,    1,      0.40],
                                [0,     0,    0,      1   ]])
#print(f"unity_pos_array_4x4: \n{unity_pos_array_4x4}")
translated_unity_pos = unity_pos_array_4x4
translated_unity_pos[:3, 3] -= unity_postion_offset  # Subtract the Unity offset to the position
print(f"translated_unity_pos: \n{translated_unity_pos}")
pos_in_robot_coordinate_system = np.dot( np.dot(unity_to_robot_rotation, unity_pos_array_4x4), unity_to_robot_rotation_inv)
#print(f"pos_in_robot_coordinate_system: \n{pos_in_robot_coordinate_system}")
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (before applying offset): \n{pos_in_robot_coordinate_system_se3}")
pos_in_robot_coordinate_system[:3, 3] += robot_postion_offset  # Add the robot coordinate system offset to the position
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (after applying offset): \n{pos_in_robot_coordinate_system_se3}")
se3_list.append(pos_in_robot_coordinate_system_se3)

""" This one does not work, probably because the matrix does not have enough precision...
unity_pos_array_4x4 = np.array([[0.47936, -0.87744, -0.01784, 0.24138],
                                [0.87642, 0.47967, -0.04260, -0.39241],
                                [0.04594, 0.00478, 0.99893, 0.30154],
                                [0,     0,    0,      1   ]])
#print(f"unity_pos_array_4x4: \n{unity_pos_array_4x4}")
translated_unity_pos = unity_pos_array_4x4
translated_unity_pos[:3, 3] -= unity_postion_offset  # Subtract the Unity offset to the position
print(f"translated_unity_pos: \n{translated_unity_pos}")
pos_in_robot_coordinate_system = np.dot( np.dot(unity_to_robot_rotation, unity_pos_array_4x4), unity_to_robot_rotation_inv)
#print(f"pos_in_robot_coordinate_system: \n{pos_in_robot_coordinate_system}")
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (before applying offset): \n{pos_in_robot_coordinate_system_se3}")
pos_in_robot_coordinate_system[:3, 3] += robot_postion_offset  # Add the robot coordinate system offset to the position
pos_in_robot_coordinate_system_se3 = SE3(pos_in_robot_coordinate_system)
print(f"pos_in_robot_coordinate_system_se3 (after applying offset): \n{pos_in_robot_coordinate_system_se3}")
se3_list.append(pos_in_robot_coordinate_system_se3)
"""

se3_list.append(neutral_pose_se3)  # Append the neutral position to close the loop in the animation

In [ ]:

# Define a list of deltas to test different positions and orientations
# eacl delta is defined by delta_x (forward), delta_y (left) and delta_z (up)
deltas = [#OK[0.00, -0.30, 0.30],
          #OK[0.00, -0.20, 0.20],
          [-0.05,  0.00, 0.00],
          [-0.05, -0.20, 0.00],
          [-0.05, -0.20, 0.10],
          [ 0.05, -0.20, 0.10],
          [ 0.05, -0.00, 0.10],
          [ 0.05, -0.00, 0.00],
          [ 0.00,  0.00, 0.00],
          ]

solutions_list = [] # List of joint angle solutions
start_pose = neutral_pose_with_bent_elbow_q
solutions_list.append(start_pose)  # Store the starting pose for later use

for index, wanted_pose in enumerate(se3_list):
    #wanted_pose = SE3.Trans(0.328, -0.076, 0.113) * SE3.OA([0, 1, 0], [0, 0, 1]) # Same as starting pose, to test sanity of IK solution and model.
    #wanted_pose = SE3.Trans(0.328, -0.076, 0.113) * SE3.OA([1, 1, 0], [0, 0, 1]) # Rotates the end-effector 45 degrees around the z-axis

    mask_priority_1 = np.array([4, 4, 4, 1, 2, 3])  # We want to prioritize the position over the orientation in the IK solution, 
                                                  # and for orientation we don't prioritize rotation around the X axis (should be along the gripper "fingers" so not too important).
                                                  # The Y and Z axes are more important for orientation, and we want to prioritize the Z axis (controlled by wrist) over 
                                                  # the Y axis (controlled by elbow and shoulder forward).
    mask_priority_2 = np.array([2, 2, 2, 0, 1, 1])  # We want to prioritize the position over the orientation in the IK solution, 
                                                # and for orientation we don't prioritize rotation around the X axis (should be along the gripper "fingers" so not too important).
    mask_list = [mask_priority_1, mask_priority_2]  # List of masks to try

    for mask_priority in mask_list:
        print(f"\n\nTrying mask priority: {mask_priority} for wanted pose #{index + 1}:\n{wanted_pose}")
        # Solve IK (inverse kinematics):
        joint_angles_solution = robot_arm.ik_LM(Tep=wanted_pose, q0=start_pose, mask=mask_priority, joint_limits=True, ilimit=300, slimit=1000)
        solution_q, is_success, iterations, num_searches, residual = joint_angles_solution
        print(f"LM Inverse kinematics solution #{index}: {'SUCCESS' if is_success else 'FAILURE'}")
        #print(f"Solution found in {iterations} iterations")
        #print(f"Number of searches performed: {num_searches}")
        #print(f"Final residual error: {residual:.6f}")
        print(f"Joint angles suggested (degrees): {np.round(np.rad2deg(solution_q))}")
        #print(f"Rotation around the Z, Y and Z axes for the wanted_pose: {wanted_pose.eul(unit='deg')}")
        #solution_se3 = robot_arm.fkine(solution_q)
        #print(f"Rotation around the Z, Y and Z axes for the solution:    {solution_se3.eul(unit='deg')}")
        if is_success:
            break  # If the solution was successful, no need to try other methods

        joint_angles_solution = robot_arm.ik_NR(Tep=wanted_pose, q0=start_pose, mask=mask_priority, joint_limits=True, ilimit=300, slimit=1000, pinv=True)
        solution_q, is_success, iterations, num_searches, residual = joint_angles_solution
        print(f"NR Inverse kinematics solution #{index}: {'SUCCESS' if is_success else 'FAILURE'}")
        print(f"Joint angles suggested (degrees): {np.round(np.rad2deg(solution_q))}")
        if is_success:
            break  # If the solution was successful, no need to try other methods

        joint_angles_solution = robot_arm.ik_GN(Tep=wanted_pose, q0=start_pose, mask=mask_priority, joint_limits=True, ilimit=300, slimit=1000, pinv=True)
        solution_q, is_success, iterations, num_searches, residual = joint_angles_solution
        print(f"GN Inverse kinematics solution #{index}: {'SUCCESS' if is_success else 'FAILURE'}")
        print(f"Joint angles suggested (degrees): {np.round(np.rad2deg(solution_q))}")
        if is_success:
            break  # If the solution was successful, no need to try other methods


    solutions_list.append(solution_q)  # Store the solution for later use

    # Check if solution respects joint limits
    q_deg = np.rad2deg(solution_q)
    within_limits = True
    for i, (q_val, limits) in enumerate(zip(q_deg, joint_limits_deg)):
        if q_val < limits[0] or q_val > limits[1]:
            print(f"WARNING: Joint {i+1} angle {q_val:.1f}° exceeds limits [{limits[0]}, {limits[1]}]")
            within_limits = False
    if within_limits:
        #print("All joint angles are within specified limits.")
        pass

    #print(f"Wanted pose: \n{wanted_pose}")
    #print(f"Solution: \n{robot_arm.fkine(solution_q)}")

    # We can animate a path from the ready pose (qr configuration) to the new wanted position and orientation
    qt = rtb.jtraj(start_pose, solution_q, 50)
    robot_arm.plot(qt.q, backend='pyplot', movie='robot_arm_vid_' + str(index) + '.gif')

    start_pose = solution_q # Update the start pose for the next iteration



solutions_list.append(neutral_pose_with_bent_elbow_q)  # Store the starting pose again at the end
print("\nList of joint angle solutions (in degrees), for easy copying into another Python program:")
solutions_array = np.round(np.rad2deg(solutions_list)).astype(int)
print("[")
for i, solution in enumerate(solutions_array):
    solution_str = "[" + ", ".join(map(str, solution)) + "]"
    if i < len(solutions_array) - 1:
        solution_str += ","
    print(f"    {solution_str}")
print("]")


# Combine all created gifs into one gif file:
print("\nCombining all animated GIFs into one...")
number_of_deltas = len(deltas)
all_frames = []
for i in range(number_of_deltas):
    # Read all frames from each animated GIF
    gif_frames = imageio.mimread(f'robot_arm_vid_{i}.gif')
    all_frames.extend(gif_frames)  # Add all frames to the combined list
imageio.mimsave('robot_arm_combined.gif', all_frames, duration=0.1)  # Save as a single gif with 0.1 second delay between frames
print("Finished combining all animated GIFs into one.")

